<a href="https://colab.research.google.com/github/nargo0o/Advanced-Machine-Learning/blob/main/Task3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
data = pd.read_csv('3.1 - Лист1.csv', sep=',')

In [7]:
data.head()

,Food,Price,Taste,Demand
0,Pizza,2,4,No
1,Pizza,5,5,No
2,Pizza,4,5,Yes
3,Pizza,1,5,Yes
4,Pizza,3,5,No


In [8]:
X = data.drop('Demand', axis = 1)
y = np.array([int(i == 'Yes') for i in data.Demand])

In [10]:
X.head()

,Food,Price,Taste
0,Pizza,2,4
1,Pizza,5,5
2,Pizza,4,5
3,Pizza,1,5
4,Pizza,3,5


In [11]:
y

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1])

In [12]:
# Рассчитайте начальную энтропию отклика
def get_H(p):
    p = np.array(p)
    return sum(-p*np.log2(p))

p = np.array([sum(y)/len(y), 1-sum(y)/len(y)])
H = get_H(p)
np.round(H, 3)

0.971

In [13]:
# Probabilities
# (Demand|Pizza), (Demand|SUSHI), (Demand|Burgers)
food_index = {'Pizza':0, 
              'Sushi':1, 
              'Burgers':2}


probs_food = np.array([[0, 0],
                       [0, 0],
                       [0, 0]])
food_type_prob = np.array([0, 0, 0])
                       
for f, d in zip(X.Food, y):
    food_type_prob[food_index[f]] += 1
    probs_food[food_index[f]][d] += 1


for food in food_index:
    print(f'(Demand|{food}):')
    print(f'\tYes: {probs_food[food_index[food]][1]}/{sum(probs_food[food_index[food]])}')
    print(f'\tNo:  {probs_food[food_index[food]][0]}/{sum(probs_food[food_index[food]])}')

# for the future
probs_food = probs_food.astype(float)
for i in range(len(probs_food)):
    probs_food[i] /= sum(probs_food[i])
    
food_type_prob = food_type_prob.astype(float)
food_type_prob /= sum(food_type_prob)

(Demand|Pizza):
	Yes: 2/5
	No:  3/5
(Demand|Sushi):
	Yes: 2/8
	No:  6/8
(Demand|Burgers):
	Yes: 4/7
	No:  3/7


In [14]:
# Вычислите условные энтропии эксперимента  при условии, 
# что выбрана пицца, суши или бургер.
food_h = []
for pp, food in zip(probs_food, food_index):
    food_h.append(get_H(pp))
    print(f'H({food}): {np.round(food_h[-1], 3)}')

H(Pizza): 0.971
H(Sushi): 0.811
H(Burgers): 0.985


In [15]:
print(food_type_prob)

[0.25 0.4  0.35]


In [16]:
# IG(Demand|Food)
H - sum(food_h * food_type_prob)

0.05887184844536031

In [17]:
def count_entropy_IG(x, y):
    
    # Class probability
    def get_H(p):
        p = np.array(p)
        return sum(-p*np.log2(p))
    p_aprior = np.array([sum(y)/len(y), 1-sum(y)/len(y)])
    H = get_H(p_aprior)
    
    # Probabilities
    index_counter = 0
    indexes = {}
    # P(Demand|class_0), P(Demand|class_1), ...
    yes_no_probs = []
    # P(class_0), P(class_1), ...
    class_prob = []

    for cur_class, d in zip(x, y):
        if not cur_class in indexes:
            indexes.update({cur_class:index_counter})
            class_prob.append(0)
            yes_no_probs.append([0, 0])
            index_counter += 1
        class_prob[indexes[cur_class]] += 1
        yes_no_probs[indexes[cur_class]][d] += 1
    
    yes_no_probs = np.array(yes_no_probs)
    class_prob = np.array(class_prob)

    for food in indexes:
        print(f'(Demand|{food}):')
        print(f'\tYes: {yes_no_probs[indexes[food]][1]}/{sum(yes_no_probs[indexes[food]])}')
        print(f'\tNo:  {yes_no_probs[indexes[food]][0]}/{sum(yes_no_probs[indexes[food]])}')

    # for the future
    yes_no_probs = yes_no_probs.astype(float)
    for i in range(len(yes_no_probs)):
        yes_no_probs[i] /= sum(yes_no_probs[i])

    class_prob = class_prob.astype(float)
    class_prob /= sum(class_prob)
    
    # H(Demand|class)
    class_H = []
    for pp, cur_class in zip(yes_no_probs, indexes):
        class_H.append(get_H(pp))
        print(f'H({cur_class}): {np.round(class_H[-1], 3)}')
    print(class_prob, class_H)
    # IG(Demand|Class)
    print('\nINFORMATION GAIN')
    print(np.round(H - sum(class_H * class_prob), 3))

In [18]:
count_entropy_IG(X.Food, y)


(Demand|Pizza):
	Yes: 2/5
	No:  3/5
(Demand|Sushi):
	Yes: 2/8
	No:  6/8
(Demand|Burgers):
	Yes: 4/7
	No:  3/7
H(Pizza): 0.971
H(Sushi): 0.811
H(Burgers): 0.985
[0.25 0.4  0.35] [0.9709505944546686, 0.8112781244591328, 0.9852281360342515]

INFORMATION GAIN
0.059


In [19]:
count_entropy_IG(X.Price, y)


(Demand|2):
	Yes: 1/5
	No:  4/5
(Demand|5):
	Yes: 2/5
	No:  3/5
(Demand|4):
	Yes: 2/3
	No:  1/3
(Demand|1):
	Yes: 2/4
	No:  2/4
(Demand|3):
	Yes: 1/3
	No:  2/3
H(2): 0.722
H(5): 0.971
H(4): 0.918
H(1): 1.0
H(3): 0.918
[0.25 0.25 0.15 0.2  0.15] [0.7219280948873623, 0.9709505944546686, 0.9182958340544896, 1.0, 0.9182958340544896]

INFORMATION GAIN
0.072


In [20]:
count_entropy_IG(X.Taste, y)

(Demand|4):
	Yes: 2/4
	No:  2/4
(Demand|5):
	Yes: 3/7
	No:  4/7
(Demand|3):
	Yes: 1/3
	No:  2/3
(Demand|2):
	Yes: 1/4
	No:  3/4
(Demand|1):
	Yes: 1/2
	No:  1/2
H(4): 1.0
H(5): 0.985
H(3): 0.918
H(2): 0.811
H(1): 1.0
[0.2  0.35 0.15 0.2  0.1 ] [1.0, 0.9852281360342515, 0.9182958340544896, 0.8112781244591328, 1.0]

INFORMATION GAIN
0.026
